In [1]:
import torch
from cephdataset import *

In [2]:
from train.pretrain import *
from train.pretrain import get_args
from mltool.universal_model_util import get_model_para_detail
import numpy as np
import os

In [3]:
ckpt_path = "checkpoints/WeathBench7066PatchDataset/PatchWrapper-AFNONet/time_step_2_pretrain-2D70N_every_1_step_random_dataset/11_11_05_03_43-seed_76545"
args=get_args(os.path.join(ckpt_path,"config.json"))

In [4]:
args.use_wandb=0
args.gpu = args.local_rank = gpu  = local_rank = 0
##### parse args: dataset_kargs / model_kargs / train_kargs  ###########
args= parse_default_args(args)
SAVE_PATH = get_ckpt_path(args)
SAVE_PATH = "debug"
args.SAVE_PATH = str(SAVE_PATH)
#args.pretrain_weight = os.path.join(args.SAVE_PATH,'pretrain_latest.pt')
########## inital log ###################
logsys = create_logsys(args,False)


if args.distributed:
    if args.dist_url == "env://" and args.rank == -1:
        args.rank = int(os.environ["RANK"])
    if args.multiprocessing_distributed:
        # For multiprocessing distributed training, rank needs to be the
        # global rank among all the processes
        args.rank = args.rank * ngpus_per_node + local_rank
    logsys.info(f"start init_process_group,backend={args.dist_backend}, init_method={args.dist_url},world_size={args.world_size}, rank={args.rank}")
    dist.init_process_group(backend=args.dist_backend, init_method=args.dist_url,world_size=args.world_size, rank=args.rank)

model           = build_model(args)
#param_groups    = timm.optim.optim_factory.add_weight_decay(model, args.weight_decay)
optimizer,lr_scheduler,criterion = build_optimizer(args,model)
loss_scaler     = torch.cuda.amp.GradScaler(enabled=True)
logsys.info(f'use lr_scheduler:{lr_scheduler}')

2022-12-01 11:43:38,912 model args: img_size= (5, 5)
2022-12-01 11:43:38,912 model args: patch_size= 1


log at debug
wandb id: None
wandb is off, the recorder list is  ['tensorboard'], we pass wandb


2022-12-01 11:43:39,458 use model ==> PatchWrapper
2022-12-01 11:43:39,460 Rank: 0, Local_rank: 0 | Number of Parameters: 46439577, Number of Buffers: 0, Size of Model: 177.1529 MB



for 2D patch model, the img_size will be force set (32,64)


2022-12-01 11:43:41,070 use lr_scheduler:<timm.scheduler.cosine_lr.CosineLRScheduler object at 0x7fbd05ef8eb0>


In [5]:
pretrain_path = os.path.join(ckpt_path,"pretrain_latest.pt")
args.pretrain_weight = pretrain_path

In [6]:
logsys.info(f"loading weight from {args.pretrain_weight}")
start_epoch, start_step, min_loss = load_model(model.module if args.distributed else model, optimizer, lr_scheduler, loss_scaler, path=args.pretrain_weight, 
                    only_model= (args.mode=='fourcast') or (args.mode=='finetune' and not args.continue_train) ,loc = 'cuda:{}'.format(args.gpu))
if args.more_epoch_train:
    assert args.pretrain_weight
    print(f"detect more epoch training, we will do a copy processing for {args.pretrain_weight}")
    os.system(f'cp {args.pretrain_weight} {args.pretrain_weight}-epoch{start_epoch}')
logsys.info("done!")

2022-12-01 11:43:44,411 loading weight from checkpoints/WeathBench7066PatchDataset/PatchWrapper-AFNONet/time_step_2_pretrain-2D70N_every_1_step_random_dataset/11_11_05_03_43-seed_76545/pretrain_latest.pt


loading model from checkpoints/WeathBench7066PatchDataset/PatchWrapper-AFNONet/time_step_2_pretrain-2D70N_every_1_step_random_dataset/11_11_05_03_43-seed_76545/pretrain_latest.pt...........


2022-12-01 11:43:44,872 done!


loading model weight success...........
loading optimizer weight success...........
loading lr_scheduler weight success...........
loading loss_scaler weight success...........
loading model success...........


In [ ]:
#args.dataset_kargs['cross_sample']=args.cross_sample=0

In [ ]:
args.dataset_kargs['batch_size']=args.batch_size = 1

In [ ]:
args.dataset_kargs['cross_sample']=args.cross_sample=0

In [7]:
train_dataset, val_dataset, train_dataloader,val_dataloader = get_train_and_valid_dataset(args,
               train_dataset_tensor=None,train_record_load=None,
               valid_dataset_tensor=None,valid_record_load=None)
logsys.info(f"use dataset ==> {train_dataset.__class__.__name__}")
logsys.info(f"Start training for {args.epochs} epochs")
master_bar = logsys.create_master_bar(args.epochs)
accu_list = ['valid_loss']
metric_dict = logsys.initial_metric_dict(accu_list)

use dataset in datasets/weatherbench_6hour
load data from datasets/weatherbench_6hour/train.npy
notice we will use around_index(28, 64, 2, 5, 5) to patch data
use dataset in datasets/weatherbench_6hour
load data from datasets/weatherbench_6hour/valid.npy


2022-12-01 11:44:06,858 use dataset ==> WeathBench7066PatchDataset
2022-12-01 11:44:06,859 Start training for 320 epochs


notice we will use around_index(28, 64, 2, 5, 5) to patch data


In [8]:
train_dataset.img_shape

(32, 64)

In [ ]:
test_data = torch.arange(32*64).reshape(32,64)

In [9]:
train_dataset.around_index.shape

(28, 64, 2, 5, 5)

In [ ]:
x = train_dataset.around_index[:,:,0]
y = train_dataset.around_index[:,:,1]

In [ ]:
args.dataset_kargs['cross_sample']=args.cross_sample=1
train_dataset.cross_sample  =1
train_dataloader  = torch.utils.data.DataLoader(train_dataset,1)

In [ ]:
train_dataset.cross_sample  =0
train_dataloader  = torch.utils.data.DataLoader(train_dataset,1)

In [ ]:
# val_dataset.cross_sample  =0 
# val_dataloader = torch.utils.data.DataLoader(val_dataset,256)

In [ ]:
epoch = 0
start_step = 0
data_loader = train_dataloader
status = 'valid'
if status == 'train':
    model.train()
    logsys.train()
elif status == 'valid':
    model.eval()
    logsys.eval()
else:
    raise NotImplementedError
accumulation_steps = model.accumulation_steps # should be 16 for finetune. but I think its ok.
half_model = next(model.parameters()).dtype == torch.float16

data_cost  = []
train_cost = []
rest_cost  = []
now = time.time()

Fethcher   = RandomSelectPatchFetcher if( status =='train' and \
                                          data_loader.dataset.use_offline_data and \
                                          data_loader.dataset.split=='train' and \
                                          'Patch' in data_loader.dataset.__class__.__name__) else Datafetcher
device     = next(model.parameters()).device
prefetcher = Fethcher(data_loader,device)
#raise
batches    = len(data_loader)

inter_b    = logsys.create_progress_bar(batches,unit=' img',unit_scale=data_loader.batch_size)
gpu        = dist.get_rank() if hasattr(model,'module') else 0

if start_step == 0:optimizer.zero_grad()
intervel = batches//100 + 1


total_diff,total_num  = torch.Tensor([0]).to(device), torch.Tensor([0]).to(device)
nan_count = 0
Nodeloss1 = Nodeloss2 = Nodeloss12 = -1

inter_b.lwrite(f"load everything, start_{status}ing......", end="\r")
preds = []
reals = []
while inter_b.update_step():
    #if inter_b.now>10:break
    step = inter_b.now
    batch = prefetcher.next()
    #print(batch[0].shape)
    #raise
    if step < start_step:continue
    #batch = data_loader.dataset.do_normlize_data(batch)

    batch = make_data_regular(batch,half_model)
    break
    

In [ ]:
model.eval()
i = 1
start = batch[0:model.history_length]
end= batch[i]
dataset = data_loader.dataset

In [ ]:
with torch.no_grad():
    
    time_stamp = None
    pos = None
    assert len(start)==1
    if isinstance(start[-1],list):
        assert len(start[-1])<=3 # only allow tensor + time_stamp + pos
        tensor, time_stamp, pos = start[-1]
        start = [tensor]

    Field  = start[-1]
    normlized_Field_list = dataset.do_normlize_data([start])[0]  #always use normlized input
    normlized_Field    = normlized_Field_list[0] if len(normlized_Field_list)==1 else torch.stack(normlized_Field_list,2)


    if time_stamp is not None or pos is not None:
        target = dataset.do_normlize_data([end[0]])[0] #always use normlized target
    else:
        target = dataset.do_normlize_data([end])[0] #always use normlized target

    if model.training and model.input_noise_std and i==1:
        normlized_Field += torch.randn_like(normlized_Field)*model.input_noise_std

    if time_stamp is not None or pos is not None :
        out   = model(normlized_Field,time_stamp,pos)
    else:
        out   = model(normlized_Field)

    extra_loss = 0
    extra_info_from_model_list = []
    if isinstance(out,(list,tuple)):
        extra_loss                 = out[1]
        extra_info_from_model_list = out[2:]
        out = out[0]

    ltmv_pred = dataset.inv_normlize_data([out])[0]

    if isinstance(start[0],(list,tuple)):
        start = start[1:]+[[ltmv_pred, 0 , end[-1]]]
    else:
        start     = start[1:] + [ltmv_pred]
    #print(ltmv_pred.shape,torch.std_mean(ltmv_pred))
    #print(target.shape,torch.std_mean(target))
    get_center_index_depend_on = model.module.get_center_index_depend_on if hasattr(model,'module') else model.get_center_index_depend_on
    if len(ltmv_pred.shape)>2: #(B,P,Z,H,W)
        if ltmv_pred.shape!=target.shape: # (B,P,W,H) -> (B,P,W-4,H) mode
            if len(target.shape) == 5:
                img_shape = target.shape[-3:]
                sld_shape = ltmv_pred.shape[-3:]
                z_idx, h_idx, l_idx = get_center_index_depend_on(sld_shape, img_shape)[0]
                target = target[...,z_idx, h_idx, l_idx]
            elif len(target.shape) == 4:
                img_shape = target.shape[-2:]
                sld_shape = ltmv_pred.shape[-2:]
                h_idx, l_idx = get_center_index_depend_on(sld_shape, img_shape)[0]
                target = target[...,h_idx, l_idx]
            else:
                raise NotImplementedError
        else:
            # (B,P,W,H) -> (B,P,W,H) mode
            pass
    else: #(B, P)
        if len(target.shape) == 4:
            B,P,W,H=target.shape
            target = target[...,W//2,H//2]
        elif len(target.shape) == 5:
            B,P,Z,W,H=target.shape
            target = target[...,Z//2,W//2,H//2]  
        else:
            raise NotImplementedError

In [10]:
def run_one_iter(model, batch, criterion, status, gpu, dataset):
    iter_info_pool={}
    loss = 0
    diff = 0
    random_run_step = np.random.randint(1,len(batch)) if len(batch)>1 else 0
    time_step_1_mode=False
    if len(batch) == 1 and isinstance(batch[0],(list,tuple)) and len(batch[0])>1:
        batch = batch[0] # (Field, FieldDt)
        time_step_1_mode=True
    if model.history_length > len(batch):
        print(f"you want to use history={model.history_length}")
        print(f"but your input batch(timesteps) only has len(batch)={len(batch)}")
        raise
    pred_step = 0
    start = batch[0:model.history_length] # start must be a list
    ltmv_pred_record = []
    target_record = []
    for i in range(model.history_length,len(batch)):# i now is the target index
        ltmv_pred, target, extra_loss, extra_info_from_model_list, start = once_forward(model,i,start,batch[i],dataset,time_step_1_mode)
        if extra_loss !=0:
            iter_info_pool[f'{status}_extra_loss_gpu{gpu}_timestep{i}'] = extra_loss.item()
        for extra_info_from_model in extra_info_from_model_list:
            for name, value in extra_info_from_model.items():
                iter_info_pool[f'valid_on_{status}_{name}_timestep{i}'] = value
        
        ltmv_pred = dataset.do_normlize_data([ltmv_pred])[0]
        print(torch.mean((ltmv_pred - target)**2))
        abs_loss = criterion(ltmv_pred,target)
        print(abs_loss)
        ltmv_pred_record.append(ltmv_pred)
        target_record.append(target)
        iter_info_pool[f'{status}_abs_loss_gpu{gpu}_timestep{i}'] =  abs_loss.item()
        pred_step+=1
        
        loss += abs_loss + extra_loss
        diff += abs_loss
        if model.random_time_step_train and i >= random_run_step:
            break
    # loss = loss/(len(batch) - 1)
    # diff = diff/(len(batch) - 1)
    loss = loss/pred_step
    diff = diff/pred_step
    return loss, diff, iter_info_pool,torch.cat(ltmv_pred_record),torch.cat(target_record)

In [ ]:
train_dataset.cross_sample  =1
train_dataloader  = torch.utils.data.DataLoader(train_dataset,256)

In [13]:
val_dataset.cross_sample  =1
val_dataloader  = torch.utils.data.DataLoader(val_dataset,256)

In [11]:
model.accumulation_steps = 1

In [14]:
epoch = 0
start_step = 0
data_loader = val_dataloader
status = 'valid'
if status == 'train':
    model.train()
    logsys.train()
elif status == 'valid':
    model.eval()
    logsys.eval()
else:
    raise NotImplementedError
accumulation_steps = model.accumulation_steps # should be 16 for finetune. but I think its ok.
half_model = next(model.parameters()).dtype == torch.float16

data_cost  = []
train_cost = []
rest_cost  = []
now = time.time()

Fethcher   = RandomSelectPatchFetcher if( status =='train' and \
                                          data_loader.dataset.use_offline_data and \
                                          data_loader.dataset.split=='train' and \
                                          'Patch' in data_loader.dataset.__class__.__name__) else Datafetcher
device     = next(model.parameters()).device
prefetcher = Fethcher(data_loader,device)
#raise
batches    = len(data_loader)

inter_b    = logsys.create_progress_bar(batches,unit=' img',unit_scale=data_loader.batch_size)
gpu        = dist.get_rank() if hasattr(model,'module') else 0

if start_step == 0:optimizer.zero_grad()
intervel = batches//100 + 1


total_diff,total_num  = torch.Tensor([0]).to(device), torch.Tensor([0]).to(device)
nan_count = 0
Nodeloss1 = Nodeloss2 = Nodeloss12 = -1

inter_b.lwrite(f"load everything, start_{status}ing......", end="\r")
preds = []
reals = []
while inter_b.update_step():
    #if inter_b.now>10:break
    step = inter_b.now
    batch = prefetcher.next()
    #print(batch[0].shape)
    #raise
    if step < start_step:continue
    #batch = data_loader.dataset.do_normlize_data(batch)

    batch = make_data_regular(batch,half_model)
    
    
    #if len(batch)==1:batch = batch[0] # for Field -> Field_Dt dataset
    data_cost.append(time.time() - now);now = time.time()
    if status == 'train':
        if hasattr(model,'set_step'):model.set_step(step=step,epoch=epoch)
        if hasattr(model,'module') and hasattr(model.module,'set_step'):model.module.set_step(step=step,epoch=epoch)
        if model.train_mode =='pretrain':
            time_truncate = max(min(epoch//3,data_loader.dataset.time_step),2)
            batch=batch[:model.history_length -1 + time_truncate]

        # the normal initial method will cause numerial explore by using timestep > 4 senenrio.
        if model.use_amp:
            with torch.cuda.amp.autocast():
                loss, abs_loss, iter_info_pool =run_one_iter(model, batch, criterion, 'train', gpu, data_loader.dataset)
        else:
            loss, abs_loss, iter_info_pool =run_one_iter(model, batch, criterion, 'train', gpu, data_loader.dataset)
        loss, nan_count, skip = nan_diagnose_weight(model,loss,nan_count,logsys)
        if skip:continue
        loss /= accumulation_steps



        if model.use_amp:
            loss_scaler.scale(loss).backward()

        else:
            loss.backward()

        if optimizer.grad_modifier is not None:
            #assert not model.use_amp
            assert accumulation_steps == 1
            if model.use_amp:
                loss_scaler.unscale_(optimizer) # do unscaler here for right gradient modify like clip or norm
            assert len(batch)==2 # we now only allow one 
            assert isinstance(batch[0],torch.Tensor)
            optimizer.grad_modifier.backward(model, batch[0], batch[1], strict=False)
            Nodeloss1, Nodeloss12, Nodeloss2 = optimizer.grad_modifier.inference(model, batch[0], batch[1], strict=False)

        #GradientModifier().backward(model,x,y)
        #nan_count, skip = nan_diagnose_grad(model,nan_count,logsys)
        # if skip:
        #     optimizer.zero_grad()
        #     continue

        if model.clip_grad:
            if model.use_amp:
                assert accumulation_steps == 1
                loss_scaler.unscale_(optimizer)
            nn.utils.clip_grad_norm_(model.parameters(), model.clip_grad)

        if (step+1) % accumulation_steps == 0:
            if model.use_amp:

                loss_scaler.step(optimizer)
                loss_scaler.update()
            else:
                optimizer.step()
            optimizer.zero_grad()
    else:
        with torch.no_grad():
            print([t.shape for t in batch])
            loss, abs_loss, iter_info_pool,pred,real =run_one_iter(model, batch, criterion, status, gpu, data_loader.dataset)
            if optimizer.grad_modifier is not None:
                Nodeloss1, Nodeloss12, Nodeloss2 = optimizer.grad_modifier.inference(model, batch[0], batch[1], strict=False)
    iter_info_pool={}
    iter_info_pool[f'{status}_loss_gpu{gpu}']     =  loss.item()
    iter_info_pool[f'{status}_Nodeloss1_gpu{gpu}'] = Nodeloss1
    iter_info_pool[f'{status}_Nodeloss12_gpu{gpu}'] = Nodeloss12
    iter_info_pool[f'{status}_Nodeloss2_gpu{gpu}'] = Nodeloss2
    total_diff  += abs_loss.item()
    #total_num   += len(batch) - 1 #batch 
    total_num   += 1 

    train_cost.append(time.time() - now);now = time.time()
    time_step_now = len(batch)
    outstring=(f"epoch:{epoch:03d} iter:[{step:5d}]/[{len(data_loader)}] [TimeLeng]:{time_step_now:} GPU:[{gpu}] abs_loss:{abs_loss.item():.2f} loss:{loss.item():.2f} cost:[Date]:{np.mean(data_cost):.1e} [Train]:{np.mean(train_cost):.1e} ")
    #print(data_loader.dataset.record_load_tensor.mean().item())
    data_cost  = []
    train_cost = []
    rest_cost = []
    inter_b.lwrite(outstring, end="\r")
    preds.append(pred.detach().cpu())
    reals.append(real.detach().cpu())

[torch.Size([256, 70, 5, 5]), torch.Size([256, 70, 5, 5])]
tensor(0.3339, device='cuda:0')
tensor(0.3339, device='cuda:0')
[torch.Size([256, 70, 5, 5]), torch.Size([256, 70, 5, 5])].33 loss:0.33 cost:[Date]:7.0e-01 [Train]:5.7e-02 
tensor(0.3240, device='cuda:0')
tensor(0.3240, device='cuda:0')
[torch.Size([256, 70, 5, 5]), torch.Size([256, 70, 5, 5])].32 loss:0.32 cost:[Date]:6.6e-01 [Train]:5.7e-02 
tensor(0.3351, device='cuda:0')
tensor(0.3351, device='cuda:0')
[torch.Size([256, 70, 5, 5]), torch.Size([256, 70, 5, 5])].34 loss:0.34 cost:[Date]:6.0e-01 [Train]:5.7e-02 
tensor(0.3363, device='cuda:0')
tensor(0.3363, device='cuda:0')
[torch.Size([256, 70, 5, 5]), torch.Size([256, 70, 5, 5])].34 loss:0.34 cost:[Date]:5.9e-01 [Train]:5.7e-02 
tensor(0.3417, device='cuda:0')
tensor(0.3417, device='cuda:0')
[torch.Size([179, 70, 5, 5]), torch.Size([179, 70, 5, 5])].34 loss:0.34 cost:[Date]:5.9e-01 [Train]:5.7e-02 
tensor(0.3168, device='cuda:0')
tensor(0.3168, device='cuda:0')


In [ ]:
# for epoch in mastera_bar:
#     if epoch < start_epoch:continue
#     if hasattr(model,'set_epoch'):model.set_epoch(epoch=epoch,epoch_total=args.epochs)
#     if hasattr(model,'module') and hasattr(model.module,'set_epoch'):model.module.set_epoch(epoch=epoch,epoch_total=args.epochs)
#     logsys.record('learning rate',optimizer.param_groups[0]['lr'],epoch, epoch_flag='epoch')
#     train_loss = run_one_epoch(epoch, start_step, model, criterion, train_dataloader, optimizer, loss_scaler,logsys,'train')


NameError: name 'master_bar' is not defined

In [ ]:
train_dataset_tensor=None;
train_record_load=None;
valid_dataset_tensor=None;
valid_record_load=None

In [ ]:
# =======================> start training <==========================
print(f"entering {args.mode} training in {next(model.parameters()).device}")
now_best_path = SAVE_PATH / 'backbone.best.pt'
latest_ckpt_p = SAVE_PATH / 'pretrain_latest.pt'


train_dataset, val_dataset, train_dataloader,val_dataloader = get_train_and_valid_dataset(args,
               train_dataset_tensor=train_dataset_tensor,train_record_load=train_record_load,
               valid_dataset_tensor=valid_dataset_tensor,valid_record_load=valid_record_load)
logsys.info(f"use dataset ==> {train_dataset.__class__.__name__}")
logsys.info(f"Start training for {args.epochs} epochs")
metric_list = ['loss']
master_bar        = logsys.create_master_bar(args.epochs)
master_bar.set_multiply_graph(figsize=(9,3),engine=[['plot','plot']],labels=[metric_list])
for epoch in master_bar:
    if epoch < start_epoch:continue
    if hasattr(model,'set_epoch'):model.set_epoch(epoch=epoch,epoch_total=args.epochs)
    if hasattr(model,'module') and hasattr(model.module,'set_epoch'):model.module.set_epoch(epoch=epoch,epoch_total=args.epochs)
    logsys.record('learning rate',optimizer.param_groups[0]['lr'],epoch)
    train_loss = run_one_epoch(epoch, start_step, model, criterion, train_dataloader, optimizer, loss_scaler,logsys,'train')
    if (not args.more_epoch_train) and (lr_scheduler is not None):lr_scheduler.step(epoch)
    #torch.cuda.empty_cache()
    #train_loss = single_step_evaluate(train_dataloader, model, criterion,epoch,logsys,status='train') if 'small' in args.train_set else -1
    val_loss   = run_one_epoch(epoch, start_step, model, criterion, val_dataloader, optimizer, loss_scaler,logsys,'valid')

    if (not args.distributed) or (args.rank == 0 and local_rank == 0) :
        logsys.info(f"Epoch {epoch} | Train loss: {train_loss:.6f}, Val loss: {val_loss:.6f}")
        logsys.record('train', train_loss, epoch)
        logsys.record('valid', val_loss, epoch)
        if use_wandb:wandb.log({"epoch":epoch,'train':train_loss,'valid':val_loss})
        if val_loss < min_loss:
            min_loss = val_loss
            if epoch > args.epochs//10:
                logsys.info(f"saving best model ....")
                save_model(model, path=now_best_path, only_model=True)
                logsys.info(f"done;")
            #if last_best_path is not None:os.system(f"rm {last_best_path}")
            #last_best_path= now_best_path
            logsys.info(f"The best accu is {val_loss}")
        logsys.record('best_loss', min_loss, epoch)
        update_experiment_info(experiment_hub_path,epoch,args)
        if epoch>args.save_warm_up:
            logsys.info(f"saving latest model ....")
            save_model(model, epoch+1, 0, optimizer, lr_scheduler, loss_scaler, min_loss, latest_ckpt_p)
            logsys.info(f"done ....")

if os.path.exists(now_best_path) and args.do_final_fourcast:
    logsys.info(f"we finish training, then start test on the best checkpoint {now_best_path}")
    start_epoch, start_step, min_loss = load_model(model.module if args.distributed else model, path=now_best_path, only_model=True)
    run_fourcast(args, model,logsys)
if use_wandb:wandb.finish()

In [ ]:
if local_rank == 0:
    print(f"Start training for {args.epochs} epochs")

master_bar        = logsys.create_master_bar(args.epochs)
last_best_path = None
for epoch in master_bar:
    if epoch < start_epoch:continue
    train_one_epoch(epoch, start_step, model, criterion, train_dataloader, optimizer, loss_scaler,lr_scheduler, min_loss,logsys)
    lr_scheduler.step(epoch)
    #torch.cuda.empty_cache()
    train_loss = single_step_evaluate(train_dataloader, model, criterion,epoch,logsys)
    #train_loss = -1
    val_loss   = single_step_evaluate(val_dataloader, model, criterion,epoch,logsys)

    if rank == 0 and local_rank == 0:
        print(f"Epoch {epoch} | Train loss: {train_loss:.6f}, Val loss: {val_loss:.6f}")
        logsys.record('train', train_loss, epoch)
        logsys.record('valid', val_loss, epoch)
        if val_loss < min_loss:
            min_loss = val_loss
            print(f"saving best model ....")
            now_best_path = SAVE_PATH / f'backbone.best.pt'
            if epoch>args.save_warm_up:save_model(model, path=now_best_path, only_model=True)
            #if last_best_path is not None:os.system(f"rm {last_best_path}")
            #last_best_path= now_best_path
            print(f"done; the best accu is {val_loss}")
        logsys.record('best_loss', min_loss, epoch)
        

In [ ]:
if __name__ == '__main__':
    args = get_args()
    ngpus = ngpus_per_node = torch.cuda.device_count()
    args.world_size = -1
    args.dist_file  = None
    args.rank       = 0
    args.dist_backend = "nccl"
    args.multiprocessing_distributed = ngpus>1
    if not hasattr(args,'train_set'):args.train_set='large'
    ip = os.environ.get("MASTER_ADDR", "127.0.0.1")
    port = os.environ.get("MASTER_PORT", "54247")
    hosts = int(os.environ.get("WORLD_SIZE", "1"))  # number of nodes
    rank = int(os.environ.get("RANK", "0"))  # node id
    gpus = torch.cuda.device_count()  # gpus per node
    args.dist_url = f"tcp://{ip}:{port}"
    if args.world_size == -1 and "SLURM_NPROCS" in os.environ:
        args.world_size = int(os.environ["SLURM_NPROCS"])
        args.rank       = int(os.environ["SLURM_PROCID"])
        jobid           = os.environ["SLURM_JOBID"]

        hostfile        = "dist_url." + jobid  + ".txt"
        if args.dist_file is not None:
            args.dist_url = "file://{}.{}".format(os.path.realpath(args.dist_file), jobid)
        elif args.rank == 0:
            import socket
            ip = socket.gethostbyname(socket.gethostname())
            port = find_free_port()
            args.dist_url = "tcp://{}:{}".format(ip, port)
            #with open(hostfile, "w") as f:f.write(args.dist_url)
        else:
            import os
            import time
            while not os.path.exists(hostfile):
                time.sleep(1)
            with open(hostfile, "r") as f:
                args.dist_url = f.read()
        print("dist-url:{} at PROCID {} / {}".format(args.dist_url, args.rank, args.world_size))
    else:
        args.world_size = 1
    args.distributed = args.world_size > 1 or args.multiprocessing_distributed
    train_dataset_tensor=valid_dataset_tensor=None

    print("======== loading data ==========")
    if 'small' in args.train_set:
        if not args.fourcast:
            train_dataset_tensor = load_small_dataset_in_memory('train').share_memory_()
            valid_dataset_tensor = load_small_dataset_in_memory('valid').share_memory_()
        else:
            train_dataset_tensor = load_small_dataset_in_memory('test').share_memory_()
            valid_dataset_tensor = None
    else:
        if args.fourcast:
            train_dataset_tensor = load_test_dataset_in_memory(years=[2018],root="/nvme/zhangtianning/datasets/ERA5").share_memory_()
            valid_dataset_tensor = None
    print("=======done==========")
    print(train_dataset_tensor.shape)
    if args.multiprocessing_distributed:
        args.world_size = ngpus_per_node * args.world_size
        torch.multiprocessing.spawn(main_worker, nprocs=ngpus_per_node, args=(ngpus_per_node, args,train_dataset_tensor,valid_dataset_tensor))
    else:
        main_worker(0, ngpus_per_node, args,train_dataset_tensor,valid_dataset_tensor)